In [74]:
import pandas as pd
import seaborn as sns
import pandas_gbq
import pydata_google_auth
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.cloud import bigquery
from tqdm.notebook import tqdm

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]
credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,
)

%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Base Tables

In [75]:
def gen_delivered_query(report): 
    return f"""
            CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Delivered_v4_{report}` AS -- everyone who have received push in the month

            SELECT  distinct b.aid AS adobe_tracking_id
            FROM
            (
                SELECT  distinct aid
                FROM
                (
                    SELECT  COALESCE(canvasName,campaignName) AS canvas_campaign_name
                        ,identity                          AS other_7
                    FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazePushNotificationContactView`
                    WHERE DATE(TIMESTAMP(eventTimestamp), 'America/New_York') BETWEEN @report_start_date AND @report_end_date
                    GROUP BY  1,2
                    HAVING (SUM(CASE WHEN eventName = 'Push Notification Sends' THEN 1 ELSE 0 END) >= 1) AND (SUM(CASE WHEN eventName = 'Push Notification Bounces' THEN 1 ELSE 0 END) = 0)
                ) a
                INNER JOIN `nbcu-sdp-sandbox-prod.sl_sandbox.Braze_Id_Adobe_Id_Map` map
                ON map.bid = a.other_7
            ) b
            -- exclude users put IN holdout group
            LEFT JOIN
            (
                SELECT  DISTINCT TrackingId AS aid
                FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
                WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER))
                AND Hold_Out_Type_Current = 'Owned Push Notification Holdout'
                AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date 
            ) g
            ON g.aid = b.aid
            WHERE g.aid is null
            """

In [76]:
def gen_holdout_query(report): 
    return f"""
            CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Holdout_v4_{report}` AS

            SELECT  DISTINCT TrackingId AS adobe_tracking_id
            FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
            WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
            AND Hold_Out_Type_Current = 'Owned Push Notification Holdout'
            AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date
            """

In [77]:
def gen_qual_query(report): 
    return f"""
            CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Channel_Qualifier_v4_{report}` AS
            
            -- webhook push opt-in canvas in Braze 
            SELECT  distinct adobe_tracking_id
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
            WHERE canvas_id = 'f4f21b32-e2ce-493f-a4dd-9132e45c65ff' --canvas_name = 'Push Optins' not displayed
            AND event_date BETWEEN @report_end_date AND DATE_ADD(@report_end_date, INTERVAL 2 DAY) --edited
            AND event_name = 'Webhook Sends'
            """

### Output Base Tables

In [78]:
def gen_aud_query(report): 
    return f"""
            CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Measurement_Audience_v4_{report}` AS

            SELECT  distinct delivered_and_holdout.adobe_tracking_id AS aid
                ,cohort
                ,user.account_type
                ,user.account_tenure
                ,user.tenure_paid_lens
                ,abandon_maa.primary_device_name as primary_device
            FROM (
                SELECT *, 'Push_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Delivered_v4_Jan23`
                UNION ALL
                SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Holdout_v4_Jan23`
            ) delivered_and_holdout

            -- Include only those who received push in the current reporting period or are in holdout
            INNER JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Channel_Qualifier_v4_Jan23` qualified
            ON delivered_and_holdout.adobe_tracking_id = qualified.adobe_tracking_id

            -- take out all abandon MAAs
            INNER JOIN
                (
                    SELECT  DISTINCT adobe_tracking_id, primary_device_name
                    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
                    WHERE report_date = @report_end_date
                    AND date_of_last_view IS NOT NULL
                ) abandon_maa
            ON delivered_and_holdout.adobe_tracking_id = abandon_maa.adobe_tracking_id

            --add attribute: account_type at the end of the reporting period
            INNER JOIN
                (
                    SELECT  DISTINCT adobe_tracking_id
                        ,account_type
                        ,account_tenure
                        ,tenure_paid_lens
                    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
                    WHERE report_date = @report_end_date 
                ) user
            ON delivered_and_holdout.adobe_tracking_id = user.adobe_tracking_id
            """

In [79]:
def gen_viewing_query(report): 
    return f"""
            CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Video_Viewing_v4_{report}` AS

            SELECT 
                adobe_tracking_id
                , COUNT (DISTINCT CASE WHEN VIDEO.num_views_started = 1 THEN video.adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts -- num_views_started is a flag
                , SUM (VIDEO.num_views_started ) AS Total_Content_Starts
                , SUM(VIDEO.num_seconds_played_no_ads)/3600 AS Viewing_Time
                , COUNT(DISTINCT CASE WHEN VIDEO.num_views_started = 1 THEN session_id ELSE NULL END) AS Distinct_Viewing_Sessions 
                , COUNT(DISTINCT(CASE WHEN (num_seconds_played_no_ads > CASE WHEN lower(consumption_type) = 'virtual channel' THEN 299 ELSE 0 END)
                        and (num_views_started>0) THEN CASE WHEN (lower(consumption_type) = "shortform") THEN "Shortform"
                                                            WHEN lower(franchise) != 'other' THEN franchise ELSE display_name
                                                            END
                        end)) as Repertoire_Pavo_Method
                        
            FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Measurement_Audience_v4_{report}` a

            INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO
                ON VIDEO.adobe_tracking_id = a.aid
                AND adobe_date between @report_start_date AND @report_end_date
            GROUP BY 1
            """

## Metric Definitions

In [80]:
def gen_base_query(report): 
    return f"""
            CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_KPI_Base_v4_{report}` AS 

            WITH
            Save_Base AS ( --'Lapsing_Users'
                SELECT distinct adobe_tracking_id, date_of_last_view
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
                WHERE report_date BETWEEN @report_start_date AND @report_end_date 
                AND days_since_last_view BETWEEN 15 AND 29 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
            )
            , Save_Denom as (
                SELECT distinct adobe_tracking_id
                FROM Save_Base
            )
            , Save_Num as (
                SELECT distinct a.adobe_tracking_id
                FROM Save_Base a
                INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO 
                    ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
                    AND adobe_date BETWEEN @report_start_date AND @report_end_date
                    AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 15 day) AND DATE_ADD(date_of_last_view, INTERVAL 29 day)
                    AND VIDEO.num_views_started > 0
            )
            , Winback_Base AS ( --'Lapsed_Users'
                SELECT distinct adobe_tracking_id, date_of_last_view
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
                WHERE report_date BETWEEN @report_start_date AND @report_end_date 
                AND days_since_last_view BETWEEN 30 AND 90 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
            )
            , Winback_Denom AS (
                SELECT distinct adobe_tracking_id
                FROM Winback_Base
            )
            , Winback_Num AS (
                SELECT distinct a.adobe_tracking_id
                FROM Winback_Base a
                INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`  VIDEO 
                    ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
                    AND VIDEO.adobe_date BETWEEN @report_start_date AND @report_end_date 
                    AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 30 day) AND DATE_ADD(date_of_last_view, INTERVAL 90 DAY)
                    AND VIDEO.num_views_started > 0
            )
            , Upgrade_Denom AS (
            -- upgrade metric 2.0: nonpaying to paying
                SELECT distinct adobe_tracking_id 
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
                WHERE paying_account_flag = 'NonPaying' 
                AND USER.report_date BETWEEN @report_start_date and @report_end_date
            )
            , Upgrade_Num AS (
            -- upgrades from premium to premium+ would not be in the denom. What even is an upgrade - this definition differs from upgrade_rank table. TODO: Align with stakeholders to define upgrades
                SELECT  distinct adobe_tracking_id
                FROM
                    (SELECT  
                            report_date
                        , adobe_tracking_id     
                    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
                    WHERE entitlement_change_flag IN ('Upgrade: Free to Premium'
                                                    , 'Upgrade: Free to Premium+')
                                                    -- , 'Upgrade: Premium to Premium+'
                    AND paying_account_flag = 'Paying'                                            
                        AND USER.report_date BETWEEN @report_start_date and @report_end_date
                    ) 
            )    
            , Cancel_Save_Denom AS (
                SELECT distinct adobe_tracking_id
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
                WHERE paying_account_flag = 'Paying'
                AND auto_renew_flag = 'OFF'
                AND report_date BETWEEN @report_start_date and @report_end_date
            )
            , Cancel_Save_Num AS (
                SELECT  distinct adobe_tracking_id
                FROM
                (
                    SELECT  adobe_tracking_id
                        ,report_date
                        ,auto_renew_flag                                                                      AS auto_renew_flag_today
                        ,LEAD(auto_renew_flag,1) OVER ( partition by adobe_tracking_id ORDER BY report_date ) AS auto_renew_flag_next_day
                    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
                    WHERE report_date BETWEEN @report_start_date AND @report_end_date
                    ORDER BY 1, 2 
                )
                WHERE auto_renew_flag_today = 'OFF'
                AND auto_renew_flag_next_day = 'ON'
            )
            , New_Upgrade_Denom AS (
                SELECT  distinct adobe_tracking_id
                FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never`
                WHERE report_date BETWEEN @report_start_date AND @report_end_date
            )
            , New_Upgrade_Num AS (
                SELECT  distinct adobe_tracking_id
                FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
                WHERE upgrade_row_number = 1
                AND report_date BETWEEN @report_start_date AND @report_end_date
            )
            , Paid_Winbacks_Denom AS (
                SELECT  distinct adobe_tracking_id
                FROM
                (
                    SELECT  adobe_tracking_id
                        ,report_date
                    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
                    WHERE paying_account_flag = 'NonPaying'
                    AND report_date BETWEEN @report_start_date AND @report_end_date 
                )
                WHERE adobe_tracking_id NOT IN ( SELECT distinct adobe_tracking_id FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never` WHERE report_date BETWEEN @report_start_date AND @report_end_date)
            )
            -- Super upgrades - paying base, from prem to prem+
            , Paid_Winbacks_Num AS (
                SELECT  distinct adobe_tracking_id
                FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
                WHERE upgrade_row_number > 1
                AND report_date BETWEEN @report_start_date AND @report_end_date
            )
            -- this metric uses silver_churn, which is based on data in PAVO dash Churn Trend
            , EOM_Paid_Churn_Denom AS (
                SELECT  distinct adobe_tracking_id
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
                WHERE base_date = @report_end_date
                AND entitlement = 'Paid'
            )
            , EOM_Paid_Churn_Num AS (
                SELECT  distinct adobe_tracking_id
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
                WHERE base_date = @report_end_date
                AND entitlement = 'Paid'
                AND Churn_flag = 'Churn'
            )
            SELECT  @report_start_date                                                              AS Report_Month
                ,a.aid
                ,a.cohort
                
                ,a.Account_Type
                ,a.Account_Tenure
                ,a.tenure_paid_lens
                
                ,CASE WHEN video.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END                 AS Viewer
                ,video.Viewing_Time
                ,video.Repertoire_Pavo_Method
                ,video.Distinct_Viewing_Sessions
                ,CASE WHEN Winback_Denom.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END         AS Winback_Denom
                ,CASE WHEN Winback_Num.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END           AS Winback_Num
                ,CASE WHEN Save_Denom.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END            AS Save_Denom
                ,CASE WHEN Save_Num.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END              AS Save_Num
                ,CASE WHEN Upgrade_Denom.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END         AS Upgrade_Denom
                ,CASE WHEN Upgrade_Num.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END           AS Upgrade_Num
                ,CASE WHEN New_Upgrade_Denom.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END     AS New_Upgrade_Denom
                ,CASE WHEN New_Upgrade_Num.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END       AS New_Upgrade_Num
                ,CASE WHEN Paid_Winbacks_Denom.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END   AS Paid_Winbacks_Denom
                ,CASE WHEN Paid_Winbacks_Num.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END     AS Paid_Winbacks_Num
                ,CASE WHEN Cancel_Save_Denom.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END     AS Cancel_Save_Denom
                ,CASE WHEN Cancel_Save_Num.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END       AS Cancel_Save_Num
                ,CASE WHEN EOM_Paid_Churn_Denom.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END  AS EOM_Paid_Churn_Denom
                ,CASE WHEN EOM_Paid_Churn_Num.adobe_tracking_id IS NOT NULL THEN 1 ELSE 0 END    AS EOM_Paid_Churn_Num
            FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Measurement_Audience_v4_{report}` a
            LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_Video_Viewing_v4_{report}` video
                ON a.aid = video.adobe_tracking_id
            LEFT JOIN Winback_Denom
                ON a.aid = Winback_Denom.adobe_tracking_id
            LEFT JOIN Winback_Num
                ON Winback_Denom.adobe_tracking_id = Winback_Num.adobe_tracking_id
            LEFT JOIN Save_Denom
                ON a.aid = Save_Denom.adobe_tracking_id
            LEFT JOIN Save_Num
                ON Save_Denom.adobe_tracking_id = Save_Num.adobe_tracking_id
            LEFT JOIN Upgrade_Denom
                ON a.aid = Upgrade_Denom.adobe_tracking_id
            LEFT JOIN Upgrade_Num 
                ON Upgrade_Denom.adobe_tracking_id = Upgrade_Num.adobe_tracking_id
            LEFT JOIN New_Upgrade_Denom
                on a.aid = New_Upgrade_Denom.adobe_tracking_id
            LEFT JOIN New_Upgrade_Num
                on New_Upgrade_Denom.adobe_tracking_id = New_Upgrade_Num.adobe_tracking_id
            LEFT JOIN Paid_Winbacks_Denom
                on a.aid = Paid_Winbacks_Denom.adobe_tracking_id
            LEFT JOIN Paid_Winbacks_Num
                on Paid_Winbacks_Num.adobe_tracking_id = Paid_Winbacks_Denom.adobe_tracking_id
            LEFT JOIN Cancel_Save_Denom
                on a.aid = Cancel_Save_Denom.adobe_tracking_id
            LEFT JOIN Cancel_Save_Num
                on Cancel_Save_Denom.adobe_tracking_id = Cancel_Save_Num.adobe_tracking_id
            LEFT JOIN EOM_Paid_Churn_Denom
                on a.aid = EOM_Paid_Churn_Denom.adobe_tracking_id
            LEFT JOIN EOM_Paid_Churn_Num
                on EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id
            """

# Backfiller

In [81]:
params = {
    'Oct22': {
        'report_start_date': '2022-10-01',
        'report_end_date': '2022-10-31'
    },
    'Nov22': {
        'report_start_date': '2022-11-01',
        'report_end_date': '2022-11-30'
    },
    'Dec22': {
        'report_start_date': '2022-12-01',
        'report_end_date': '2022-12-31'
    },
    'Jan23': {
        'report_start_date': '2023-01-01',
        'report_end_date': '2023-01-31'
    },
    'Feb23': {
        'report_start_date': '2023-02-01',
        'report_end_date': '2023-02-28'
    },
    'Mar23': {
        'report_start_date': '2023-03-01',
        'report_end_date': '2023-03-31'
    }
}

In [82]:
gen_queries = [
    gen_delivered_query,
    gen_holdout_query,
    gen_qual_query,
    gen_aud_query,
    gen_viewing_query,
    gen_base_query,
]

In [83]:
def build(report, dates, queries):
    client = bigquery.Client()
    queries = [fn(report) for fn in queries]
    for q in queries:
        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("report_start_date", "STRING", dates['report_start_date']),
                bigquery.ScalarQueryParameter("report_end_date", "STRING", dates['report_end_date']),
            ]
        )
        job = client.query(q, job_config=job_config)
        job.result()
    print('Done', report)

In [84]:
threads = []
results = []

with ThreadPoolExecutor(len(params)) as executor:
    for report, dates in params.items():
        threads.append(executor.submit(build, report, dates, gen_queries))
    for future in as_completed(threads):
        results.append(future.result())

In [ ]:
client = bigquery.Client()
query = 'CREATE or replace TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.push_channel_kpi` AS ' + \
            (' UNION ALL ').join([f'(select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_KPI_Base_v4_{report}`)' for report in params])

client.query(query).result()

# Analysis

In [ ]:
def gen_analysis_query_total(report): 
    return f"""
            WITH 
            CTE_1 AS (
                SELECT  @report_start_date                                                           AS Report_Month
                --, Account_Type
                    ,COUNT(distinct CASE WHEN cohort = 'Push_Targeted' THEN aid END )            AS Distinct_Cohort_Size_Targeted
                    ,COUNT(distinct CASE WHEN cohort = 'Holdout' THEN aid END)                    AS Distinct_Cohort_Size_Holdout
                    
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN viewer END )                   AS Total_Returns_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN viewer END)                           AS Total_Returns_Holdout
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Viewing_Time END )              AS Total_Usage_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Viewing_Time END)                      AS Total_Usage_Holdout
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Repertoire_Pavo_Method END )    AS Total_Repertoire_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Repertoire_Pavo_Method END)            AS Total_Repertoire_Holdout
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Distinct_Viewing_Sessions END ) AS Total_Viewing_Sessions_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Distinct_Viewing_Sessions END)         AS Total_Viewing_Sessions_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Winback_Denom END )             AS Winback_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Winback_Num END)                AS Winback_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Denom END )                    AS Winback_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Num END)                       AS Winback_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Save_Denom END )                AS Save_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Save_Num END)                   AS Save_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Denom END )                       AS Save_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Num END)                          AS Save_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Upgrade_Denom END )             AS Upgrades_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Upgrade_Num END)                AS Upgrades_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Denom END )                    AS Upgrades_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Num END)                       AS Upgrades_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN New_Upgrade_Denom END )         AS Total_New_Upgrade_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN New_Upgrade_Num END)            AS Total_New_Upgrade_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Denom END )                AS Total_New_Upgrade_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Num END)                   AS Total_New_Upgrade_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Paid_Winbacks_Denom END )       AS Total_Paid_Winbacks_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Paid_Winbacks_Num END)          AS Total_Paid_Winbacks_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Denom END )              AS Total_Paid_Winbacks_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Num END)                 AS Total_Paid_Winbacks_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Cancel_Save_Denom END )         AS Total_Cancel_Save_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Cancel_Save_Num END)            AS Total_Cancel_Save_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Denom END )                AS Total_Cancel_Save_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Num END)                   AS Total_Cancel_Save_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN EOM_Paid_Churn_Denom END )      AS EOM_Paid_Churn_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN EOM_Paid_Churn_Num END)         AS EOM_Paid_Churn_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END )             AS EOM_Paid_Churn_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)                AS EOM_Paid_Churn_Num_Holdout
                
                FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_KPI_Base_v4_{report}`
                GROUP BY  1 --, 2

            ), CTE_2 AS (
                SELECT  Report_Month
                        -- ,Account_Type
                        ,Distinct_Cohort_Size_Targeted                                                            AS Push_Engagers
                        ,Distinct_Cohort_Size_Holdout                                                             AS Push_Holdout
                        ,safe_divide(Total_Returns_Targeted,Distinct_Cohort_Size_Targeted)                        AS Return_Rate_Engagers
                        ,safe_divide(Total_Returns_Holdout,Distinct_Cohort_Size_Holdout)                          AS Return_Rate_Holdout
                        ,safe_divide(Total_Usage_Targeted,Distinct_Cohort_Size_Targeted)                          AS Usage_Engagers
                        ,safe_divide(Total_Usage_Holdout,Distinct_Cohort_Size_Holdout)                            AS Usage_Holdout
                        ,safe_divide(Total_Repertoire_Targeted,Distinct_Cohort_Size_Targeted)                     AS Repertoire_Engagers
                        ,safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)                       AS Repertoire_Holdout
                        ,safe_divide(Total_Viewing_Sessions_Targeted,Distinct_Cohort_Size_Targeted)               AS Sessions_Engagers
                        ,safe_divide(Total_Viewing_Sessions_Holdout,Distinct_Cohort_Size_Holdout)                 AS Sessions_Holdout
                        ,Winback_Denom_Targeted
                        ,safe_divide(Winback_Num_Targeted,Winback_Denom_Targeted)                                 AS Winback_Rate_Engagers
                        ,safe_divide(Winback_Num_Holdout,Winback_Denom_Holdout)                                   AS Winback_Rate_Holdout
                        ,Save_Denom_Targeted
                        ,safe_divide(Save_Num_Targeted,Save_Denom_Targeted)                                       AS Save_Rate_Engagers
                        ,safe_divide(Save_Num_Holdout,Save_Denom_Holdout)                                         AS Save_Rate_Holdout
                        ,Upgrades_Denom_Targeted
                        ,safe_divide(Upgrades_Num_Targeted,Upgrades_Denom_Targeted)                               AS Upgrade_Rate_Engagers
                        ,safe_divide(Upgrades_Num_Holdout,Upgrades_Denom_Holdout)                                 AS Upgrade_Rate_Holdout
                        ,Total_New_Upgrade_Denom_Targeted
                        ,safe_divide(Total_New_Upgrade_Num_Targeted,Total_New_Upgrade_Denom_Targeted)             AS New_Upgrade_Rate_Engagers
                        ,safe_divide(Total_New_Upgrade_Num_Holdout,Total_New_Upgrade_Denom_Holdout)               AS New_Upgrade_Rate_Holdout
                        ,Total_Paid_Winbacks_Denom_Targeted
                        ,safe_divide(Total_Paid_Winbacks_Num_Targeted,Total_Paid_Winbacks_Denom_Targeted)         AS Paid_Winback_Rate_Engagers
                        ,safe_divide(Total_Paid_Winbacks_Num_Holdout,Total_Paid_Winbacks_Denom_Holdout)           AS Paid_Winback_Rate_Holdout
                        ,Total_Cancel_Save_Denom_Targeted
                        ,safe_divide(Total_Cancel_Save_Num_Targeted,Total_Cancel_Save_Denom_Targeted)             AS Cancel_Save_Rate_Engagers
                        ,safe_divide(Total_Cancel_Save_Num_Holdout,Total_Cancel_Save_Denom_Holdout)               AS Cancel_Save_Rate_Holdout
                        ,EOM_Paid_Churn_Denom_Targeted
                        ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted)                   AS Paid_Churn_Rate_Engagers
                        ,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)                     AS Paid_Churn_Rate_Holdout
                FROM CTE_1
            )

            SELECT  Report_Month
                --,Account_Type
                ,Push_Engagers
                ,Push_Holdout

                ,Return_Rate_Engagers                                                                                        AS Return_Rate_Engagers
                ,Return_Rate_Holdout                                                                                         AS Return_Rate_Holdout
                ,Return_Rate_Engagers - Return_Rate_Holdout                                                                  AS Return_Rate_Lift_PTS
                ,safe_divide(Return_Rate_Engagers,Return_Rate_Holdout) *100                                                  AS Return_Rate_Lift_Index
                ,(Return_Rate_Engagers - Return_Rate_Holdout) * Push_Engagers                                               AS Returns_Incrementals

                ,Usage_Engagers                                                                                              AS Usage_Engagers
                ,Usage_Holdout                                                                                               AS Usage_Holdout
                ,Usage_Engagers - Usage_Holdout                                                                              AS Usage_Lift_PTS
                ,safe_divide(Usage_Engagers,Usage_Holdout) *100                                                              AS Usage_Lift_Index
                ,(Usage_Engagers - Usage_Holdout) * Push_Engagers                                                           AS Usage_Incrementals

                ,Repertoire_Engagers                                                                                         AS Repertoire_Engagers
                ,Repertoire_Holdout                                                                                          AS Repertoire_Holdout
                ,Repertoire_Engagers - Repertoire_Holdout                                                                    AS Repertoire_Lift_PTS
                ,safe_divide(Repertoire_Engagers,Repertoire_Holdout) *100                                                    AS Repertoire_Lift_Index
                ,(Repertoire_Engagers - Repertoire_Holdout) * Push_Engagers                                                 AS Repertoire_Incrementals

                ,Sessions_Engagers                                                                                           AS Sessions_Engagers
                ,Sessions_Holdout                                                                                            AS Sessions_Holdout
                ,Sessions_Engagers - Sessions_Holdout                                                                        AS Sessions_Lift_PTS
                ,safe_divide(Sessions_Engagers,Sessions_Holdout) *100                                                        AS Sessions_Lift_Index
                ,(Sessions_Engagers - Sessions_Holdout) * Push_Engagers                                                     AS Sessions_Incrementals

                ,Winback_Rate_Engagers                                                                                       AS Winback_Rate_Engagers
                ,Winback_Rate_Holdout                                                                                        AS Winback_Rate_Holdout
                ,Winback_Rate_Engagers - Winback_Rate_Holdout                                                                AS Winback_Rate_Lift_PTS
                ,safe_divide(Winback_Rate_Engagers,Winback_Rate_Holdout) *100                                                AS Winback_Rate_Lift_Index
                ,(Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted                                     AS Winback_Incrementals

                ,Save_Rate_Engagers                                                                                          AS Save_Rate_Engagers
                ,Save_Rate_Holdout                                                                                           AS Save_Rate_Holdout
                ,Save_Rate_Engagers - Save_Rate_Holdout                                                                      AS Save_Rate_Lift_PTS
                ,safe_divide(Save_Rate_Engagers,Save_Rate_Holdout) *100                                                      AS Save_Rate_Lift_Index
                ,(Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted                                              AS Save_Rate_Lift_Incrementals

                ,Upgrade_Rate_Engagers                                                                                       AS Upgrade_Rate_Engagers
                ,Upgrade_Rate_Holdout                                                                                        AS Upgrade_Rate_Holdout
                ,Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                                                                AS Upgrade_Rate_Lift_PTS
                ,safe_divide(Upgrade_Rate_Engagers,Upgrade_Rate_Holdout) *100                                                AS Upgrade_Rate_Lift_Index
                ,(Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted                                    AS Upgrade_Incrementals

                ,New_Upgrade_Rate_Engagers                                                                                   AS New_Upgrade_Rate_Engagers
                ,New_Upgrade_Rate_Holdout                                                                                    AS New_Upgrade_Rate_Holdout
                ,New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                                                        AS New_Upgrade_Rate_Lift_PTS
                ,safe_divide(New_Upgrade_Rate_Engagers,New_Upgrade_Rate_Holdout) *100                                        AS New_Upgrade_Rate_Lift_Index
                ,(New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted                   AS New_Upgrade_Incrementals

                ,Paid_Winback_Rate_Engagers                                                                                  AS Paid_Winback_Rate_Engagers
                ,Paid_Winback_Rate_Holdout                                                                                   AS Paid_Winback_Rate_Holdout
                ,Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                                                      AS Paid_Winback_Rate_Lift_PTS
                ,safe_divide(Paid_Winback_Rate_Engagers,Paid_Winback_Rate_Holdout) *100                                      AS Paid_Winback_Rate_Lift_Index
                ,(Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted               AS Paid_Winback_Rate_Lift_Incrementals

                ,Cancel_Save_Rate_Engagers                                                                             AS Cancel_Save_Rate_Engagers
                ,Cancel_Save_Rate_Holdout                                                                              AS Cancel_Save_Rate_Holdout
                ,Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout                                            AS Cancel_Save_Rate_Lift_PTS
                ,safe_divide(Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout) *100                            AS Cancel_Save_Rate_Lift_Index
                ,(Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout) * Total_Cancel_Save_Denom_Targeted     AS Cancel_Save_Rate_Incrementals

                ,Paid_Churn_Rate_Engagers                                                                                    AS EOM_Paid_Churn_Rate_Engagers
                ,Paid_Churn_Rate_Holdout                                                                                     AS EOM_Paid_Churn_Rate_Holdout
                ,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                                          AS EOM_Paid_Churn_Rate_Lift_PTS
                ,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                                          AS EOM_Paid_Churn_Rate_Lift_Index
                ,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted                        AS EOM_Paid_Churn_Rate_Incrementals

            FROM CTE_2
            """

In [ ]:
def gen_analysis_query_acc(report): 
    return f"""
            WITH 
            CTE_1 AS (
                SELECT  @report_start_date                                                           AS Report_Month
                        , Account_Type
                    ,COUNT(distinct CASE WHEN cohort = 'Push_Targeted' THEN aid END )            AS Distinct_Cohort_Size_Targeted
                    ,COUNT(distinct CASE WHEN cohort = 'Holdout' THEN aid END)                    AS Distinct_Cohort_Size_Holdout
                    
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN viewer END )                   AS Total_Returns_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN viewer END)                           AS Total_Returns_Holdout
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Viewing_Time END )              AS Total_Usage_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Viewing_Time END)                      AS Total_Usage_Holdout
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Repertoire_Pavo_Method END )    AS Total_Repertoire_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Repertoire_Pavo_Method END)            AS Total_Repertoire_Holdout
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Distinct_Viewing_Sessions END ) AS Total_Viewing_Sessions_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Distinct_Viewing_Sessions END)         AS Total_Viewing_Sessions_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Winback_Denom END )             AS Winback_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Winback_Num END)                AS Winback_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Denom END )                    AS Winback_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Num END)                       AS Winback_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Save_Denom END )                AS Save_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Save_Num END)                   AS Save_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Denom END )                       AS Save_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Num END)                          AS Save_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Upgrade_Denom END )             AS Upgrades_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Upgrade_Num END)                AS Upgrades_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Denom END )                    AS Upgrades_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Num END)                       AS Upgrades_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN New_Upgrade_Denom END )         AS Total_New_Upgrade_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN New_Upgrade_Num END)            AS Total_New_Upgrade_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Denom END )                AS Total_New_Upgrade_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Num END)                   AS Total_New_Upgrade_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Paid_Winbacks_Denom END )       AS Total_Paid_Winbacks_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Paid_Winbacks_Num END)          AS Total_Paid_Winbacks_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Denom END )              AS Total_Paid_Winbacks_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Num END)                 AS Total_Paid_Winbacks_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Cancel_Save_Denom END )         AS Total_Cancel_Save_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN Cancel_Save_Num END)            AS Total_Cancel_Save_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Denom END )                AS Total_Cancel_Save_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Num END)                   AS Total_Cancel_Save_Num_Holdout

                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN EOM_Paid_Churn_Denom END )      AS EOM_Paid_Churn_Denom_Targeted
                    ,SUM(CASE WHEN cohort = 'Push_Targeted' THEN EOM_Paid_Churn_Num END)         AS EOM_Paid_Churn_Num_Targeted
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END )             AS EOM_Paid_Churn_Denom_Holdout
                    ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)                AS EOM_Paid_Churn_Num_Holdout
                
                FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_KPI_Base_v4_{report}`
                GROUP BY  1, 2

            ), CTE_2 AS (
                SELECT  Report_Month
                        ,Account_Type
                        ,Distinct_Cohort_Size_Targeted                                                            AS Push_Engagers
                        ,Distinct_Cohort_Size_Holdout                                                             AS Push_Holdout
                        ,safe_divide(Total_Returns_Targeted,Distinct_Cohort_Size_Targeted)                        AS Return_Rate_Engagers
                        ,safe_divide(Total_Returns_Holdout,Distinct_Cohort_Size_Holdout)                          AS Return_Rate_Holdout
                        ,safe_divide(Total_Usage_Targeted,Distinct_Cohort_Size_Targeted)                          AS Usage_Engagers
                        ,safe_divide(Total_Usage_Holdout,Distinct_Cohort_Size_Holdout)                            AS Usage_Holdout
                        ,safe_divide(Total_Repertoire_Targeted,Distinct_Cohort_Size_Targeted)                     AS Repertoire_Engagers
                        ,safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)                       AS Repertoire_Holdout
                        ,safe_divide(Total_Viewing_Sessions_Targeted,Distinct_Cohort_Size_Targeted)               AS Sessions_Engagers
                        ,safe_divide(Total_Viewing_Sessions_Holdout,Distinct_Cohort_Size_Holdout)                 AS Sessions_Holdout
                        ,Winback_Denom_Targeted
                        ,safe_divide(Winback_Num_Targeted,Winback_Denom_Targeted)                                 AS Winback_Rate_Engagers
                        ,safe_divide(Winback_Num_Holdout,Winback_Denom_Holdout)                                   AS Winback_Rate_Holdout
                        ,Save_Denom_Targeted
                        ,safe_divide(Save_Num_Targeted,Save_Denom_Targeted)                                       AS Save_Rate_Engagers
                        ,safe_divide(Save_Num_Holdout,Save_Denom_Holdout)                                         AS Save_Rate_Holdout
                        ,Upgrades_Denom_Targeted
                        ,safe_divide(Upgrades_Num_Targeted,Upgrades_Denom_Targeted)                               AS Upgrade_Rate_Engagers
                        ,safe_divide(Upgrades_Num_Holdout,Upgrades_Denom_Holdout)                                 AS Upgrade_Rate_Holdout
                        ,Total_New_Upgrade_Denom_Targeted
                        ,safe_divide(Total_New_Upgrade_Num_Targeted,Total_New_Upgrade_Denom_Targeted)             AS New_Upgrade_Rate_Engagers
                        ,safe_divide(Total_New_Upgrade_Num_Holdout,Total_New_Upgrade_Denom_Holdout)               AS New_Upgrade_Rate_Holdout
                        ,Total_Paid_Winbacks_Denom_Targeted
                        ,safe_divide(Total_Paid_Winbacks_Num_Targeted,Total_Paid_Winbacks_Denom_Targeted)         AS Paid_Winback_Rate_Engagers
                        ,safe_divide(Total_Paid_Winbacks_Num_Holdout,Total_Paid_Winbacks_Denom_Holdout)           AS Paid_Winback_Rate_Holdout
                        ,Total_Cancel_Save_Denom_Targeted
                        ,safe_divide(Total_Cancel_Save_Num_Targeted,Total_Cancel_Save_Denom_Targeted)             AS Cancel_Save_Rate_Engagers
                        ,safe_divide(Total_Cancel_Save_Num_Holdout,Total_Cancel_Save_Denom_Holdout)               AS Cancel_Save_Rate_Holdout
                        ,EOM_Paid_Churn_Denom_Targeted
                        ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted)                   AS Paid_Churn_Rate_Engagers
                        ,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)                     AS Paid_Churn_Rate_Holdout
                FROM CTE_1
            )

            SELECT  Report_Month
                ,Account_Type
                ,Push_Engagers
                ,Push_Holdout

                ,Return_Rate_Engagers                                                                                        AS Return_Rate_Engagers
                ,Return_Rate_Holdout                                                                                         AS Return_Rate_Holdout
                ,Return_Rate_Engagers - Return_Rate_Holdout                                                                  AS Return_Rate_Lift_PTS
                ,safe_divide(Return_Rate_Engagers,Return_Rate_Holdout) *100                                                  AS Return_Rate_Lift_Index
                ,(Return_Rate_Engagers - Return_Rate_Holdout) * Push_Engagers                                               AS Returns_Incrementals

                ,Usage_Engagers                                                                                              AS Usage_Engagers
                ,Usage_Holdout                                                                                               AS Usage_Holdout
                ,Usage_Engagers - Usage_Holdout                                                                              AS Usage_Lift_PTS
                ,safe_divide(Usage_Engagers,Usage_Holdout) *100                                                              AS Usage_Lift_Index
                ,(Usage_Engagers - Usage_Holdout) * Push_Engagers                                                           AS Usage_Incrementals

                ,Repertoire_Engagers                                                                                         AS Repertoire_Engagers
                ,Repertoire_Holdout                                                                                          AS Repertoire_Holdout
                ,Repertoire_Engagers - Repertoire_Holdout                                                                    AS Repertoire_Lift_PTS
                ,safe_divide(Repertoire_Engagers,Repertoire_Holdout) *100                                                    AS Repertoire_Lift_Index
                ,(Repertoire_Engagers - Repertoire_Holdout) * Push_Engagers                                                 AS Repertoire_Incrementals

                ,Sessions_Engagers                                                                                           AS Sessions_Engagers
                ,Sessions_Holdout                                                                                            AS Sessions_Holdout
                ,Sessions_Engagers - Sessions_Holdout                                                                        AS Sessions_Lift_PTS
                ,safe_divide(Sessions_Engagers,Sessions_Holdout) *100                                                        AS Sessions_Lift_Index
                ,(Sessions_Engagers - Sessions_Holdout) * Push_Engagers                                                     AS Sessions_Incrementals

                ,Winback_Rate_Engagers                                                                                       AS Winback_Rate_Engagers
                ,Winback_Rate_Holdout                                                                                        AS Winback_Rate_Holdout
                ,Winback_Rate_Engagers - Winback_Rate_Holdout                                                                AS Winback_Rate_Lift_PTS
                ,safe_divide(Winback_Rate_Engagers,Winback_Rate_Holdout) *100                                                AS Winback_Rate_Lift_Index
                ,(Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted                                     AS Winback_Incrementals

                ,Save_Rate_Engagers                                                                                          AS Save_Rate_Engagers
                ,Save_Rate_Holdout                                                                                           AS Save_Rate_Holdout
                ,Save_Rate_Engagers - Save_Rate_Holdout                                                                      AS Save_Rate_Lift_PTS
                ,safe_divide(Save_Rate_Engagers,Save_Rate_Holdout) *100                                                      AS Save_Rate_Lift_Index
                ,(Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted                                              AS Save_Rate_Lift_Incrementals

                ,Upgrade_Rate_Engagers                                                                                       AS Upgrade_Rate_Engagers
                ,Upgrade_Rate_Holdout                                                                                        AS Upgrade_Rate_Holdout
                ,Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                                                                AS Upgrade_Rate_Lift_PTS
                ,safe_divide(Upgrade_Rate_Engagers,Upgrade_Rate_Holdout) *100                                                AS Upgrade_Rate_Lift_Index
                ,(Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted                                    AS Upgrade_Incrementals

                ,New_Upgrade_Rate_Engagers                                                                                   AS New_Upgrade_Rate_Engagers
                ,New_Upgrade_Rate_Holdout                                                                                    AS New_Upgrade_Rate_Holdout
                ,New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                                                        AS New_Upgrade_Rate_Lift_PTS
                ,safe_divide(New_Upgrade_Rate_Engagers,New_Upgrade_Rate_Holdout) *100                                        AS New_Upgrade_Rate_Lift_Index
                ,(New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted                   AS New_Upgrade_Incrementals

                ,Paid_Winback_Rate_Engagers                                                                                  AS Paid_Winback_Rate_Engagers
                ,Paid_Winback_Rate_Holdout                                                                                   AS Paid_Winback_Rate_Holdout
                ,Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                                                      AS Paid_Winback_Rate_Lift_PTS
                ,safe_divide(Paid_Winback_Rate_Engagers,Paid_Winback_Rate_Holdout) *100                                      AS Paid_Winback_Rate_Lift_Index
                ,(Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted               AS Paid_Winback_Rate_Lift_Incrementals

                ,Cancel_Save_Rate_Engagers                                                                             AS Cancel_Save_Rate_Engagers
                ,Cancel_Save_Rate_Holdout                                                                              AS Cancel_Save_Rate_Holdout
                ,Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout                                            AS Cancel_Save_Rate_Lift_PTS
                ,safe_divide(Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout) *100                            AS Cancel_Save_Rate_Lift_Index
                ,(Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout) * Total_Cancel_Save_Denom_Targeted     AS Cancel_Save_Rate_Incrementals

                ,Paid_Churn_Rate_Engagers                                                                                    AS EOM_Paid_Churn_Rate_Engagers
                ,Paid_Churn_Rate_Holdout                                                                                     AS EOM_Paid_Churn_Rate_Holdout
                ,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                                          AS EOM_Paid_Churn_Rate_Lift_PTS
                ,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                                          AS EOM_Paid_Churn_Rate_Lift_Index
                ,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted                        AS EOM_Paid_Churn_Rate_Incrementals

            FROM CTE_2
            ORDER BY 1,2
    """

In [ ]:
def gen_analysis_query_churn(report): 
	return f"""
			WITH 
			CTE_1 AS (
			SELECT 
				@report_start_date AS Report_Month
				--, Account_Type
				, tenure_paid_lens
				
				,COUNT(distinct CASE WHEN (cohort = 'Push_Targeted') AND (Account_Type = 'Paying SVOD') THEN aid END )            AS Distinct_Cohort_Size_Targeted
				,COUNT(distinct CASE WHEN (cohort = 'Holdout') AND (Account_Type = 'Paying SVOD') THEN aid END)                    AS Distinct_Cohort_Size_Holdout

				,SUM(CASE WHEN cohort = 'Push_Targeted' THEN EOM_Paid_Churn_Denom END )      AS EOM_Paid_Churn_Denom_Targeted
				,SUM(CASE WHEN cohort = 'Push_Targeted' THEN EOM_Paid_Churn_Num END)         AS EOM_Paid_Churn_Num_Targeted
				,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END )             AS EOM_Paid_Churn_Denom_Holdout
				,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)                AS EOM_Paid_Churn_Num_Holdout
			FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Push_KPI_Base_v4_{report}`
			GROUP BY 1,2
			), CTE_2 AS (
				SELECT Report_Month
			--, Account_Type
			, tenure_paid_lens
			, Distinct_Cohort_Size_Targeted   as Push_Engagers
			, Distinct_Cohort_Size_Holdout   as Push_Holdout

			, EOM_Paid_Churn_Denom_Targeted
			, safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
			, safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

			from CTE_1
			)

			SELECT Report_Month
			--, Account_Type
			, tenure_paid_lens
			, Push_Engagers
			, Push_Holdout

			, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
			, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
			, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
			, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
			, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals

			FROM CTE_2
			ORDER BY 1,2
			"""

In [ ]:
analyze_queries = [
    gen_analysis_query_total,
    gen_analysis_query_acc,
    gen_analysis_query_churn
]

In [ ]:
def build_report(report, dates, queries):
    queries = [fn(report) for fn in queries]
    with pd.ExcelWriter(f'push_kpi_{report}.xlsx') as writer:
        for name, q in zip(['total', 'account', 'churn'], queries):
            query_config = {
                'query': {
                    'parameterMode': 'NAMED',
                    'queryParameters': [
                        {
                            'name': 'report_start_date',
                            'parameterType': {'type': 'STRING'},
                            'parameterValue': {'value': dates['report_start_date']}
                        },
                        {
                            'name': 'report_end_date',
                            'parameterType': {'type': 'STRING'},
                            'parameterValue': {'value': dates['report_end_date']}
                        }
                    ]
                }
            }
            df = pandas_gbq.read_gbq(q, configuration=query_config)
            df.to_excel(writer, sheet_name=name)
        
    print('Done', report)

In [ ]:
threads = []
results = []

with ThreadPoolExecutor(len(params)) as executor:
    for report, dates in params.items():
        threads.append(executor.submit(build_report, report, dates, analyze_queries))
    for future in as_completed(threads):
        results.append(future.result())

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading:   0%|          |



Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|


Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading:   0%|          |Done Jan23
Downloading: 100%|██████████|
Done Mar23

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Done Oct22
Done Nov22

Downloading: 100%|██████████|
Done Dec22
Downloading: 100%|██████████|
Done Feb23
